In [15]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447"
data = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
data.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [19]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

x = data[["preco", "idade_do_modelo","km_por_ano"]]
y = data["vendido"]

SEED = 158020
np.random.seed(SEED)
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 25, stratify = y)
print("Treinamos com %d elemenos e testaremos com %d elementos" %(len(train_x), len(test_x))) 

Treinamos com 9975 elemenos e testaremos com 25 elementos


In [20]:
from sklearn.dummy import DummyClassifier
dummy_stratified = DummyClassifier()
dummy_stratified.fit(train_x, train_y)
accuracy = dummy_stratified.score(test_x, test_y) * 100

print("A acurácia do dummy statified foi de %.2f%%" % accuracy)

A acurácia do dummy statified foi de 60.00%


In [22]:
from sklearn.tree import DecisionTreeClassifier 

SEED = 158020
np.random.seed(SEED)
model = DecisionTreeClassifier(max_depth=2)
model.fit(train_x, train_y)
predictions = model.predict(test_x)
accuracy = accuracy_score(test_y, predictions) * 100
print("A accuracy foi %.2f%%" % accuracy)

A accuracy foi 76.00%


In [25]:
x = data[["preco", "idade_do_modelo","km_por_ano"]]
y = data["vendido"]

SEED = 158020
np.random.seed(SEED)
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 25, stratify = y)
print("Treinamos com %d elemenos e testaremos com %d elementos" %(len(train_x), len(test_x))) 
model = DecisionTreeClassifier(max_depth=2)
model.fit(train_x, train_y)
predictions = model.predict(test_x)
accuracy = accuracy_score(test_y, predictions) * 100
print("A accuracy foi %.2f%%" % accuracy)

Treinamos com 9975 elemenos e testaremos com 25 elementos
A accuracy foi 76.00%


Ao alterarmos o seed para 5 aumentamos nossa accuracy mas, dependendo do número que chutassemos tomariamos uma decisão baseado em uma aleatoriadade, e não faz sentido basearmos nossa decisão nessa aleatoriedade, se rodamos o teste e treino uma única vez corremos o risco
de ter separado o treino e teste de uma maneira não muito boa, então precisamos  de um algoritmo para treinar e testar mais de uma vez para ter uma estimativa.

In [31]:
def imprime_resultados(results):
    media = results['test_score'].mean()
    desvio_padrao = results['test_score'].std()
    print("Accuracy médio: %.2f" % (media * 100))
    print("Accuracy com cross validation, 5 = [%.2f, %.2f]" %((media - 2 * desvio_padrao) * 100,( media + 2 * desvio_padrao)*100))

In [35]:
from sklearn.model_selection import cross_validate

SEED = 158020
np.random.seed(SEED)
model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, x, y, cv = 5, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 75.78
Accuracy com cross validation, 5 = [75.21, 76.35]


# Aleatoriadade no cross_validation


In [37]:
from sklearn.model_selection import KFold

SEED = 158020
np.random.seed(SEED)

cv = KFold(n_splits = 10)
model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, x, y, cv = cv, return_train_score=False)
imprime_resultados(results) 

Accuracy médio: 75.78
Accuracy com cross validation, 5 = [74.37, 77.19]


In [39]:
from sklearn.model_selection import KFold

SEED = 158020
np.random.seed(SEED)

cv = KFold(n_splits = 10, shuffle = True)
model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, x, y, cv = cv, return_train_score=False)
imprime_resultados(results) 

Accuracy médio: 75.78
Accuracy com cross validation, 5 = [73.58, 77.98]


# Situação de exemplo de desbalanceamento

In [42]:
data_azar = data.sort_values("vendido", ascending=True)
x_azar = data_azar[["preco", "idade_do_modelo","km_por_ano"]]
y_azar = data_azar["vendido"]

data_azar.head()

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704


In [44]:
from sklearn.model_selection import KFold

SEED = 158020
np.random.seed(SEED)

cv = KFold(n_splits = 10)
model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results) 

Accuracy médio: 57.84
Accuracy com cross validation, 5 = [34.29, 81.39]


In [46]:
from sklearn.model_selection import KFold

SEED = 158020
np.random.seed(SEED)

cv = KFold(n_splits = 10, shuffle = True)
model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results) 

Accuracy médio: 75.76
Accuracy com cross validation, 5 = [73.02, 78.50]


In [49]:
from sklearn.model_selection import StratifiedKFold

SEED = 158020
np.random.seed(SEED)

cv = StratifiedKFold(n_splits = 10, shuffle = True)
model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results) 

Accuracy médio: 75.78
Accuracy com cross validation, 5 = [74.10, 77.46]


# Gerando dados aleatórios de carro para  de agrupamento ao usar nosso estimador

In [50]:
np.random.seed(SEED)
data['modelo'] = data.idade_do_modelo + np.random.randint(-2, 3, size = 10000)
data.modelo = data.modelo + abs(data.modelo.min()) + 1
data.head()
data.modelo.value_counts()

20    873
19    830
18    788
21    707
17    687
15    669
16    616
22    597
14    577
13    576
12    497
11    397
10    376
23    347
9     320
8     288
7     217
24    199
6     186
5     120
4      66
3      44
2      20
1       3
Name: modelo, dtype: int64

# Testando validação cruzada com GroupKFold

In [52]:
from sklearn.model_selection import GroupKFold

SEED = 158020
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, x_azar, y_azar, cv = cv, groups = data.modelo, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 75.76
Accuracy com cross validation, 5 = [72.91, 78.60]


# Cross validation com StandardScaler

In [57]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
scaler.fit(train_x)
train_x_scaler = scaler.transform(train_x)
test_x_scaler = scaler.transform(test_x)

model = SVC()
model.fit(train_x_scaler, train_y)
predictions = model.predict(test_x_scaler)

accuracy = accuracy_score(test_y, predictions) * 100
print("A accuracy foi %.2f%%" % accuracy)

A accuracy foi 76.00%


In [58]:
from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
model = SVC()
results = cross_validate(model, x_azar, y_azar, cv = cv, groups = data.modelo, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 77.24
Accuracy com cross validation, 5 = [73.71, 80.77]


In [59]:
from sklearn.pipeline import Pipeline

SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao', scaler), ('estimador', modelo)])

cv = GroupKFold(n_splits = 10)
results = cross_validate(pipeline, x_azar, y_azar, cv = cv, groups = data.modelo, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 76.66
Accuracy com cross validation, 5 = [73.65, 79.66]
